## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
#from config import g_key

from configparser import ConfigParser
config = ConfigParser()
config.read('C:/Users/zionh/config_test.cfg')

g_key = config.get('google', 'g_key')

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('C:/Users/zionh/OneDrive/Desktop/Class_Projects/Module_6_WeatherPy_Analysis/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tezu,IN,27.9167,96.1667,74.21,34,45,1.48,scattered clouds
1,1,Kamaishi,JP,39.2667,141.8833,50.95,51,35,9.69,scattered clouds
2,2,Mataura,NZ,-46.1927,168.8643,62.47,45,93,6.98,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,83.01,35,20,3.00,few clouds
4,4,Kulachi,PK,31.9286,70.4592,64.15,24,99,5.82,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Hobart,AU,-42.8794,147.3294,83.01,35,20,3.00,few clouds
7,7,Manggar,ID,-2.8833,108.2667,85.75,63,79,0.54,broken clouds
12,12,Bambous Virieux,MU,-20.3428,57.7575,82.22,68,11,8.81,few clouds
17,17,Rikitea,PF,-23.1203,-134.9692,76.44,78,6,21.05,light rain
19,19,Port Blair,IN,11.6667,92.7500,83.79,71,60,12.12,broken clouds
28,28,Airai,TL,-8.9266,125.4092,76.44,50,52,2.13,broken clouds
29,29,Faanui,PF,-16.4833,-151.7500,80.87,76,100,15.70,light rain
34,34,Avarua,CK,-21.2078,-159.7750,78.85,61,20,11.50,few clouds
35,35,Atuona,PF,-9.8000,-139.0333,79.00,76,3,22.08,clear sky
37,37,Alyangula,AU,-13.8483,136.4192,88.72,58,29,3.71,scattered clouds


In [15]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID        171
City           171
Country        171
Lat            171
Lng            171
Max Temp       171
Humidity       171
Cloudiness     171
Wind Speed     171
Description    171
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#there are no empty rows


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Hobart,AU,83.01,few clouds,-42.8794,147.3294,
7,Manggar,ID,85.75,broken clouds,-2.8833,108.2667,
12,Bambous Virieux,MU,82.22,few clouds,-20.3428,57.7575,
17,Rikitea,PF,76.44,light rain,-23.1203,-134.9692,
19,Port Blair,IN,83.79,broken clouds,11.6667,92.7500,
28,Airai,TL,76.44,broken clouds,-8.9266,125.4092,
29,Faanui,PF,80.87,light rain,-16.4833,-151.7500,
34,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
35,Atuona,PF,79.00,clear sky,-9.8000,-139.0333,
37,Alyangula,AU,88.72,scattered clouds,-13.8483,136.4192,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
        hotel_df.loc[index, "Hotel Name"] = np.nan
hotel_df.head(10)    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Hobart,AU,83.01,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
7,Manggar,ID,85.75,broken clouds,-2.8833,108.2667,Guest Hotel
12,Bambous Virieux,MU,82.22,few clouds,-20.3428,57.7575,Casa Tia Villa
17,Rikitea,PF,76.44,light rain,-23.1203,-134.9692,People ThankYou
19,Port Blair,IN,83.79,broken clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
28,Airai,TL,76.44,broken clouds,-8.9266,125.4092,Juvinal Martin
29,Faanui,PF,80.87,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
34,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
35,Atuona,PF,79.00,clear sky,-9.8000,-139.0333,Villa Enata
37,Alyangula,AU,88.72,scattered clouds,-13.8483,136.4192,Groote Eylandt Lodge


In [19]:
# 7. Drop the rows where there is no Hotel Name.
#There are no rows missing a hotel name
cleaned_hotel_df = hotel_df.dropna()


In [20]:
cleaned_hotel_df.count()

City           167
Country        167
Max Temp       167
Description    167
Lat            167
Lng            167
Hotel Name     167
dtype: int64

In [21]:
# 8a. Create the output File (CSV)

output_data_file = 'C:/Users/zionh/OneDrive/Desktop/Class_Projects/Module_6_WeatherPy_Analysis/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv

cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 

locations = cleaned_hotel_df[["Lat", "Lng"]]
max_temp = cleaned_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))